# Assignment 2: Convolutional Neural Networks and AutoML
---

# Import Required Libraries
Import the necessary libraries, including PyTorch and the chosen hyperparameter optimization library.

In [ ]:
# Importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from support import load_dataset
import matplotlib.pyplot as plt

# Importing hyperparameter optimization library
import optuna
# from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load and Preprocess the Wefabricate Dataset
Load the Wefabricate dataset and preprocess it for use with a CNN.

In [ ]:
train_data, test_data = load_dataset()

# Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
# Visualize data with their labels

# Define the class labels 
class_labels = ["Defect", "No Defect"]

# Get a batch of training data
images, labels = next(iter(train_loader))

# Create a grid of the images and print the labels
plt.figure(figsize=(15, 15))
for i in range(5):
    plt.subplot(5, 5, i + 1)
    plt.imshow(images[i][0], cmap='gray')
    plt.axis('off')
    plt.title(class_labels[labels[i]])
    
plt.show()

# Define the CNN Model
Define a CNN to classify product images in the Wefabricate dataset.

In [ ]:
# Define the CNN Model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, padding=0, stride=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(3, 3)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 8 * 3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 8 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the network
model = ConvNet()

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the CNN Model
Train the CNN model on the training set.

In [ ]:
# Move the model to the device
model.to(device)
model.train()

# Training the CNN Model
for epoch in range(1):

    running_loss = 0.0
    for i, data in tqdm(enumerate(train_loader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        outputs = outputs.squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print the loss every epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Finished Training")

# Test the CNN Model
Test the trained CNN model on the test set.

In [ ]:
# Test the model
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        predicted = torch.round(
            torch.sigmoid(outputs)
        ).squeeze()  # Ensure predicted is 1D
        labels = labels.float()  # Ensure labels are float for comparison
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

# Hyperparameter Selection
Choose five hyperparameters to tune.

# Hyperparameter Optimization with Random Search
Use random search to tune the chosen hyperparameters.

# Hyperparameter Optimization with Sophisticated Method
Use a more sophisticated method than random search to tune the chosen hyperparameters.

# 5-Fold Cross-Validation
Perform 5-fold cross-validation on the training set to calculate the validation accuracy for the hyperparameters.

# Compare Results Before and After Hyperparameter Optimization
Compare the results obtained before and after automatic hyperparameter optimization in terms of accuracy.

# Save Model Weights Before and After Hyperparameter Tuning
Save the trained model weights before and after hyperparameter tuning.

In [ ]:
# Save the model weights before hyperparameter tuning
torch.save(model.state_dict(), 'model_weights_before_tuning.pth')

# Load the model with the best parameters found by sophisticated search
best_model = ConvNet()
if best_params['optimizer'] == 'SGD':
    best_optimizer = SGD(best_model.parameters(), lr=best_params['learning_rate'], momentum=best_params['momentum'])
else:
    best_optimizer = Adam(best_model.parameters(), lr=best_params['learning_rate'])

# Train the best model
for epoch in range(best_params['num_epochs']):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        best_optimizer.zero_grad()
        outputs = best_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        best_optimizer.step()
        running_loss += loss.item()

# Save the model weights after hyperparameter tuning
torch.save(best_model.state_dict(), 'model_weights_after_tuning.pth')